# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)

        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. The Apache Cassandra Coding Portion of the Project. 

## The <font color=red>event_datafile_new.csv</font> is located within the Workspace directory and contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = {
            'class' : 'SimpleStrategy',
            'replication_factor' : 1
        }
        """
    )

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Queries are Created to Ask Three Questions of the Data

### Query 1

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4

**CREATE TABLE Statement 1**

We just need the 5 columns _sessionId, itemInSession, artist, length,_ and _song_ which are mentioned in the question. Because we are filtering the data using `WHERE sessionId = 338 AND itemInSession = 4` in the SELECT statement below, we have to choose `(sessionId, itemInSession)` as our partition key.

In [10]:
qry_create_music_app_history = """
    CREATE TABLE IF NOT EXISTS history_by_session_and_item (
        sessionId int,
        itemInSession int,
        artist text, 
        length float,
        song text,
        PRIMARY KEY ((sessionId, itemInSession))
    )
""" 
try:
    session.execute(qry_create_music_app_history)
except Exception as e:
    print(e)                

**INSERT Statement 1**

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    qry_insert_music_app_history = """
        INSERT INTO history_by_session_and_item (
            sessionId,
            itemInSession,
            artist, 
            length,
            song
        )
        VALUES (%s, %s, %s, %s, %s)
    """
    for line in csvreader:
        session.execute(
            qry_insert_music_app_history,
            (
                int(line[8]),
                int(line[3]),
                line[0],
                float(line[5]),
                line[9],
            )
        )

**SELECT Statement 1 to Answer Question 1**

In [12]:
qry_select_music_app_history = """
    SELECT
        artist,
        song,
        length
    FROM history_by_session_and_item
    WHERE sessionId = 338
        AND itemInSession = 4
"""

try:
    rows = session.execute(qry_select_music_app_history)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


So it is Faithless' Music Matters (Mark Knight Dub) that was heard during sessionId = 338 and itemInSession = 4. Its length is 495.307312 sec.

### Query 2

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Here we need the 7 columns _userId, sessionId, itemInSession, artist, firstName, lastName,_ and _song_ which are mentioned in the question. Because we are filtering the data using `WHERE userid = 10 AND sessionid = 182` in the SELECT statement below, we have to choose _(userId, sessionId)_ as our partition key. And as we sort the results by _itemInSession_ ascending, we choose it as our clustering column.

In [13]:
qry_create_music_app_history2 = """
    CREATE TABLE IF NOT EXISTS history_by_user_and_session (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        firstName text,
        lastName text,
        song text,
        PRIMARY KEY ((userid, sessionid), itemInSession)
    )
""" 
try:
    session.execute(qry_create_music_app_history2)
except Exception as e:
    print(e)
                    

**INSERT Statement 2**

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    qry_insert_music_app_history2 = """
        INSERT INTO history_by_user_and_session (
            userId,
            sessionId,
            itemInSession,
            artist,
            firstName,
            lastName,
            song
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    for line in csvreader:
        session.execute(
            qry_insert_music_app_history2,
            (
                int(line[10]),
                int(line[8]),
                int(line[3]),
                line[0],
                line[1],
                line[4],
                line[9],
            )
        )

**SELECT Statement 2 to Answer Question 2**

In [15]:
qry_select_music_app_history2 = """
    SELECT
        itemInSession,
        artist,
        song,
        firstName,
        lastName
    FROM history_by_user_and_session
    WHERE userid = 10
        AND sessionid = 182
"""

try:
    rows = session.execute(qry_select_music_app_history2)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
df

,iteminsession,artist,song,firstname,lastname
0,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,1,Three Drives,Greece 2000,Sylvie,Cruz
2,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


This are are all the song (given by artist & title) which Sylvie Cruz (userId 10) listened to in session with sessionId = 182. The correct order can be inspected in the iteminsession column.

### Query 3

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

And here in addition to the 3 columns _song, firstName,_ and _lastName_, we need the _userId_ as different users may have the same first- and lastname. AS we are filtering the data using `WHERE song = 'All Hands Against His Own'` in the SELECT statement below, we have to choose _(song)_ as our partition key. And as we sort the results by _itemInSession_ ascending, we choose it as our clustering column.

In [16]:
qry_create_music_app_history3 = """
    CREATE TABLE IF NOT EXISTS songs_by_user (
        song text,
        userId int,
        firstName text,
        lastName text,
        PRIMARY KEY ((song), userId)
    )
""" 
try:
    session.execute(qry_create_music_app_history3)
except Exception as e:
    print(e)    

**INSERT Statement 3**

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    qry_insert_music_app_history3 = """
        INSERT INTO songs_by_user (
            song,
            userId,
            firstName,
            lastName
        )
        VALUES (%s, %s, %s, %s)
    """
    for line in csvreader:
        session.execute(
            qry_insert_music_app_history3,
            (
                line[9],
                int(line[10]),
                line[1],
                line[4],
            )
        )

**SELECT Statement 3 to Answer Question 3**

In [18]:
qry_select_music_app_history3 = """
    SELECT
        firstName,
        lastName
    FROM songs_by_user
    WHERE song = 'All Hands Against His Own'
"""

try:
    rows = session.execute(qry_select_music_app_history3)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


...are the users, who listened to the song 'All Hands Against His Own' in my music app history.

### Dropping the Tables Before and Closing Out the Session and Cluster Connection

In [19]:
tables = [
    "history_by_session_and_item",
    "history_by_user_and_session",
    "songs_by_user",
]


for table in tables:
    try:
        rows = session.execute(f"DROP TABLE {table}")
    except Exception as e:
        print(e)

session.shutdown()
cluster.shutdown()